# Data Exploration for Insights

This session will guide you through various techniques for examining data to gain deeper insights, build intuition, and initiate hypothesis testing. By the end of this workshop, you'll have a robust set of data exploration skills, laying a solid foundation for any subsequent data analysis or machine learning projects.

The data used in this notebook was generated by 'DATA_datasetCompilation.ipynb'. I encourage you to review that notebook to understand the data exploration approach taken for data compilation and cleaning.

Please ensure you have the following data files stored in the 'INPUTS/' folder:
1. CQ_Data_20240604.txt
2. WtshdAttributeTable_20240603.txt

In [ ]:
# Import all libraries at the top of your code so you can easily see and organize all the packges you are using. 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
from sklearn.linear_model import LinearRegression
import os
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.cm as cm
import matplotlib.gridspec as gridspec

# Setting some global parameters based on my preferences
pd.set_option('display.max_columns', None)
plt.rcdefaults()

In [ ]:
# Set up your filepaths
inputDataFilepath = 'XXX/INPUTS/'

In [ ]:
# Loading in the concentration, flow, and attribute data.
CQ = pd.read_csv(inputDataFilepath+'CQ_Data_20240604.txt', sep=",", dtype={'SITE': str, 'HUC02': str})
WtshdAtt = pd.read_csv(inputDataFilepath+'WtshdAttributeTable_20240603.txt', sep=",", dtype={'SITE': str, 'HUC02': str})

# Merging CQ data with their watershe attributes
CQ_attrib = CQ.merge(WtshdAtt, on=['SITE','YEAR'], how='left')
CQ_attrib = CQ_attrib.dropna()

## Exploring Concentration-Discharge Relationships
Today we will be looking at nitrogen concentration (C) and discharge (Q). The relationship between solute and discharge have been explored extensively because they are useful to understand the underlying drivers of catchment export dynamics and are valuable for interpreting the processes driving export patterns of solutes.

Different typologies of CQ relationships provide integrated signals of processes hapenning within the catchment. 

Some foundational literature on CQ relationships in human-impacted landscapes:
- [Musolff, A., Schmidt, C., Selle, B., & Fleckenstein, J. H. (2015). Catchment controls on solute export. Advances in Water Resources, 86, 133–146.](https://www.sciencedirect.com/science/article/abs/pii/S030917081500233X)
- [Basu, N. B., Destouni, G., Jawitz, J. W., Thompson, S. E., Loukinova, N. V., Darracq, A., Zanardo, S., Yaeger, M., Sivapalan, M., Rinaldo, A., & Rao, P. S. C. (2010). Nutrient loads exported from managed catchments reveal emergent biogeochemical stationarity. Geophysical Research Letters, 37(23).](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2010GL045168)
- [Speir, S. L., Rose, L. A., Blaszczak, J. R., Kincaid, D. W., Fazekas, H. M., Webster, A. J., Wolford, M. A., Shogren, A. J., & Wymore, A. S. (2024). Catchment concentration–discharge relationships across temporal scales: A review. WIREs. Water, 11(2). https://doi.org/10.1002/wat2.1702](https://wires.onlinelibrary.wiley.com/doi/full/10.1002/wat2.1702)

### Understanding your watersheds
Before jumping into finding that patterns and trends we see, let's take a look at our data. We want to explore the spatial coverage and type of watersheds that are in our dataset. This step will help guide us on the type of questions or hypothesis we can ask with the data we have.

In [ ]:
# Loading in the watershed shapefile. 
wtshd = gpd.read_file(inputDataFilepath+'boundaries-shapefiles-by-aggeco/bas_all.shp')
wtshd = wtshd.rename(columns={'GAGE_ID':'SITE'})
US_boundary = gpd.read_file(inputDataFilepath+'cb_2017_us_state_5m/cb_2017_us_state_5m.shp')
wtshd_f = wtshd[wtshd['SITE'].isin(CQ_attrib['SITE'])]
US_boundary = US_boundary.to_crs(wtshd_f.crs)

In [ ]:
# Let's compare all GAGE watersheds to the dataset that has water quality samples. 

# Task: Plot the watershed shapefile and the filtered watershed shapefile. 
# Syntax: shapefile.plot(ax=[basemap], color='colorName')

# I've plotted the base map, you plot the other 2.
base = US_boundary.plot(color='white', edgecolor='black')

wtshd.plot(ax=base, color='sienna', edgecolor='none')
wtshd_f.plot(ax=base, color='darkcyan', edgecolor='none')
base.set_xlim(-3*10**6, 3*10**6)
base.set_ylim(0, 3.5*10**6)

In [ ]:
print('Number of watersheds in the dataset:',len(wtshd_f))

In [ ]:
# How many datapoints does each watershed have? What is the distribution of years?
fig, axs = plt.subplots(1,2, figsize=(10, 5))
noWtshd = CQ_attrib.groupby('SITE')['Conc'].count()
noWtshd = wtshd_f.merge(noWtshd, on='SITE')

ax1 = plt.subplot(1,2,1)
US_boundary.plot(color='white', edgecolor='black', ax=ax1)
noWtshd.plot(ax=ax1, 
             column='Conc', 
             cmap='OrRd',
             legend = True,
             vmin=0,
             vmax=100,
             edgecolor='none',
             legend_kwds={"label": "Number of observations", "orientation": "horizontal"})

# Contiguous US
base.set_xlim(-3*10**6, 3*10**6)
base.set_ylim(0, 3.5*10**6)

# What years we do we have in our dataset?
noYears = CQ_attrib.groupby('YEAR')['Conc'].count().reset_index() # Note: reset index turning the index of the Series into a column in the DataFrame
noYears.columns = ['YEAR', 'Conc_Count']

ax2 = plt.subplot(1,2,2)
sns.barplot(data=noYears, x='YEAR', y='Conc_Count', ax=ax2)
ax2.set_ylabel('Number of observations')
ax2.set_xticks(['1980', '1990', '2000', '2010', '2020'])
plt.tight_layout()

That's some decent coverage! We have watersheds across the entire contiguous US, including some of the more data sparse watersheds in the west. 

Let's look at the attributes and understanding the distribution of our data. First we will start with the univariate approach, which just means looking at one variable at a time. There are a few ways to go about this.

- '.describe()' will print out the summary statistics of your variable.
- 'boxplots' can be used to visualize the summary statistics but not the most intuitive to understand the distribution. 
- 'histogram' are great to see the distribution, but understanding the summary statistics can be a challenge.

Using multiple approaches can help get a better understanding of the data. 

In [ ]:
CQ_attrib.columns

In [ ]:
CQ_attrib['AREA_SQKM'].describe()

In [ ]:
# Generating 3 plots using the area attribute ('AREA_SQKM'). 
fig, axs = plt.subplots(1,3, figsize=(10, 3))

ax1 = plt.subplot(1,3,1)
sns.histplot(data=CQ_attrib, x="AREA_SQKM", ax=ax1)
ax1.set_ylim(0,20000)

ax2 = plt.subplot(1,3,2)
sns.histplot(x=np.log10(CQ_attrib['AREA_SQKM']), ax=ax2)
plt.tight_layout()
plt.xlabel('Log AREA SQKM')

ax3 = plt.subplot(1,3,3)
sns.boxplot(data=CQ_attrib, x='AREA_SQKM', ax=ax3)
plt.tight_layout()

In [ ]:
# Now create histograms and boxplots for 'UrbLU_pct', 'AgLU_pct, and 'NatLU_pct'

fig, axs = plt.subplots(3,2, figsize=(6, 6))

# Urban LU
ax1 = plt.subplot(3,2,1)
#sns.histplot(data=*, x=*, ax=ax1)
ax2 = plt.subplot(3,2,2)
#sns.boxplot(data=*, x=*, ax=ax2)


# Agricultural LU
ax3 = plt.subplot(3,2,3)
#sns.histplot(data=*, x=*, ax=ax3)
ax4 = plt.subplot(3,2,4)
#sns.boxplot(data=*, x=*, ax=ax4)

# Natural LU
ax3 = plt.subplot(3,2,3)
#sns.histplot(data=*, x=*, ax=ax5)
ax4 = plt.subplot(3,2,4)
#sns.boxplot(data=*, x=*, ax=ax6)

# Set your axis limits using the following syntax. ax1 refers to the first plot.
# ax1.set_ylim(0,100) 
# ax2.set_xlim(0,75) 

plt.tight_layout()

In [ ]:
# Let's look at Urban land use and wastewater treatment plant density. 
fig, axs = plt.subplots(2,2, figsize=(6, 6))

ax1 = plt.subplot(2,2,1)
sns.histplot(data=CQ_attrib, x="UrbLU_pct", ax=ax1)

ax2 = plt.subplot(2,2,2)
sns.boxplot(data=CQ_attrib, x="UrbLU_pct", ax=ax2)

# WWTPd_100km2
#ax3 = plt.subplot(2,2,3)
#sns.histplot(data=CQ_attrib, x=#, ax=ax3)

#ax4 = plt.subplot(2,2,4)
#sns.boxplot(data=CQ_attrib, x=#,, ax=ax4)

# Set your axis limits using the following syntax. ax1 refers to the first plot.
# ax1.set_xlim(0,100)
# ax2.set_xlim(0,75) 

plt.tight_layout()

In [ ]:
# Moving onto our fertilizer and manure nitrogen inputs, and tile drainage.
# FERT_kgkm2, MANU_kgkm2, TD_pct
fig, axs = plt.subplots(3,2, figsize=(6, 6))

ax1 = plt.subplot(3,2,1)
sns.histplot(data=CQ_attrib, x="FERT_kgkm2", ax=ax1)
ax1.set_ylim(0,10000)

ax2 = plt.subplot(3,2,2)
sns.boxplot(data=CQ_attrib, x="FERT_kgkm2", ax=ax2)
ax2.set_xlim(0,5000)

ax3 = plt.subplot(3,2,3)
#sns.histplot

ax4 = plt.subplot(3,2,4)
#sns.boxplot

ax5 = plt.subplot(3,2,5)
#sns.histplot

ax6 = plt.subplot(3,2,6)
#sns.boxplot
plt.tight_layout()

In [ ]:
# Lastly we will look at the static attributes
# BFI_AVE, SLOPE_pct, AI, medianGWD_m
fig, axs = plt.subplots(4,2, figsize=(6, 8))

ax1 = plt.subplot(4,2,1)
#sns.histplot

ax2 = plt.subplot(4,2,2)
#sns.boxplot

ax3 = plt.subplot(4,2,3)
#sns.histplot

ax4 = plt.subplot(4,2,4)
#sns.boxplot

ax5 = plt.subplot(4,2,5)
#sns.histplot

ax6 = plt.subplot(4,2,6)
#sns.boxplot

ax7 = plt.subplot(4,2,7)
#sns.histplot

ax8 = plt.subplot(4,2,8)
#sns.boxplot

plt.tight_layout()

In [ ]:
# Now to look at multivariate relationships, pairwise relationships we can make a pairplot of the attributes.
CQ_attrib_filtered = CQ_attrib[['LAT', 'LONG', 'UrbLU_pct', 'NatLU_pct', 'AgLU_pct', 'FERT_kgkm2',
       'MANU_kgkm2', 'BFI_AVE', 'SLOPE_pct', 'AI', 'medianGWD_m', 'TD_pct', 'WWTPd_100km2']]
sns.set(font_scale=1.5)
sns.pairplot(CQ_attrib_filtered, corner=True)

In [ ]:
corr_mat = CQ_attrib_filtered.corr().stack().reset_index(name="correlation")

# Draw each cell as a scatter point with varying size and color
g = sns.relplot(
    data=corr_mat,
    x="level_0", y="level_1", hue="correlation", size="correlation",
    palette="vlag", hue_norm=(-1, 1), edgecolor=".7",
    height=10, sizes=(50, 250), size_norm=(-.2, .8),
)

# Tweak the figure to finalize
g.set(xlabel="", ylabel="", aspect="equal")
g.despine(left=True, bottom=True)
g.ax.margins(.02)
for label in g.ax.get_xticklabels():
    label.set_rotation(90)

#### What do we see?
- Latitude and Longitude are...
- Land use and latitude/longitude are...
- Percent slope of the watershed is correlated with agricultural LU and inputs. Why?
- Agricultural land use and tile drainage are ...
- Aridity index, groundwater depth, and tile drainage are correlated.
- At this scale, urban land use (LU) and WWTP density are...
- Fertilizer and manure are correlated with WWTP density.

In [ ]:
# Now let's look at the flows and concentrations!
plt.rcdefaults() # Setting plotting properties back to default

fig, axs = plt.subplots(1,2, figsize=(6, 3))
ax1 = plt.subplot(1,2,1)
sns.histplot(x=np.log10(CQ_attrib['Q_m3s']), ax=ax1)
plt.xlabel('Log Flow rate')

ax2 = plt.subplot(1,2,2)
sns.boxplot(data=CQ_attrib, x="Q_m3s", ax=ax2)
ax2.set_xlim(0,100)

plt.tight_layout()

In [ ]:
CQ_attrib['Q_m3s'].describe()

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(6, 3))
ax1 = plt.subplot(1,2,1)
sns.histplot(x=np.log10(CQ_attrib['Conc']), ax=ax1)
plt.xlabel('Log Conc (mg-N/L)')

ax2 = plt.subplot(1,2,2)
sns.boxplot(data=CQ_attrib, x="Conc", ax=ax2)
plt.xlabel('Conc (mg-N/L)')
plt.tight_layout()

In [ ]:
CQ_attrib['Conc'].describe()

In [ ]:
# Where are the highest concentrations?

# Let's groupby 'SITE' to aggregate each site's concentation records.
meanC = CQ_attrib.groupby('SITE')['Conc'].mean().reset_index()
meanC_shp = wtshd_f.merge(meanC, on='SITE')

# Creating a map to visualize our results.
c_lim = q3 = CQ_attrib['Conc'].describe()['75%']
base = US_boundary.plot(color='grey', edgecolor='black')
meanC_shp.plot(ax=base, 
               column='Conc', 
               cmap='OrRd',
               legend = True,
               vmin=0,
               vmax=c_lim,
               edgecolor='none',
               legend_kwds={"label": "Mean Concentration (mg-N/L)", 
                            "orientation": "horizontal"})

# Contiguous US
base.set_xlim(-3*10**6, 3*10**6)
base.set_ylim(0, 3.5*10**6)

# You miss some nuance looking at the map at this scale. Change the x and y-lims to inspect a few areas. 
# Midwest
#base.set_xlim(-0.5*10**6, 1.25*10**6)
#base.set_ylim(1.5*10**6, 3*10**6)

# Western US
#base.set_xlim(-3*10**6,0.5*10**6)
#base.set_ylim(0.5*10**6, 2.25*10**6)

# Texas
#base.set_xlim(-1*10**6,0.25*10**6)
#base.set_ylim(0, 1.5*10**6)

#### What do we see?
- ...

### Developing research questions
Working with open data is faster than conducting your own data collection, but it comes with uncertainties about data availability and the insights the data can provide. The resolution, both spatial and temporal, can limit the questions you can answer. The data itself can also impose limitations on your research scope. In many of my research projects, I don't follow the traditional hypothesis-testing approach. To avoid spending excessive time trying to make my data fit my needs, I let the data guide the questions I _can_ answer.

**Summary of data:** 
- ...
  
**Questions we can ask:**
- ...

### Feature Engineering
#### Power Law Slope (β)
Feature engineering refers to the process of transforming raw data into new features (attributes) to gain better insights and enhance model performance. This term is widely used in the machine learning space, as it is a crucial step in improving model accuracy and effectiveness.

One of the features we will generate is the C-Q Slope (β):

β is the exponent in the power-law equation (C = aQ^β), where C represents concentration and Q represents flow. We can determine this slope by regressing C and Q in the log-log space.


![CQ-relationship by Speirs et al.](../INPUTS/schematics/CQ_relationship.png)

(figure recreated from Speirs et al.)

To help you understand the theoretical 'β' metric in practical terms, here are explanations of the different regimes. These are generalized and can be more nuanced in reality.

**Enrichment regime**: As flow increases, so does concentration. In this regime, excess nitrogen in the landscape is transport-limited. Thus, during a storm, the excess nitrogen is flushed and transported to the river.

**Constant regime**: As flow increases, concentration remains constant. In this regime, as runoff increases, there is sufficient nitrogen available for transport, maintaining a stable concentration. This regime often exhibits the highest concentrations due to an excess nitrogen store.

**Dilution regime**: As flow increases, concentration decreases. In this regime, there is usually a fixed mass of nitrogen, and as flows increase, this mass is diluted. For example, if a wastewater treatment plant has a constant mass loading, increased flow will dilute this load, reducing concentrations.

In [ ]:
# Let's take a look at how this looks with real data. You can choose two of your own or use '04176500' and '02096960'.
# Recall the column names: 'Q_m3s', 'Conc', 'YEAR'
fig, axs = plt.subplots(2,2, figsize=(6, 6))

# Isolating the station data.
CQ_Site = CQ_attrib.loc[CQ_attrib['SITE'] == '04176500']

# Use a scatter plot to visualize the results from  'CQ_Site'. You can use c='YEAR' to color each marker based on the year. 

# plt1 = axs[0, 0].scatter(data=*, x=*, y=*, c=*,  alpha=0.5)
# plt.colorbar(plt1, ax=axs[0, 0], orientation ='horizontal')
# Add a title using this syntax: axs[0, 0].set_title('**')

# You can also plot the log-log space by transforming Q and C (x=np.log10(CQ_Site['Q_m3s']) and y=np.log10(CQ_Site['Conc']))
#axs[0, 1].scatter(data=*, x=*, y=*, c=*,  alpha=0.5)

CQ_Site = CQ_attrib.loc[CQ_attrib['SITE'] == '02096960']

# Plotting another station
plt2 = axs[1, 0].scatter(data=CQ_Site, x='Q_m3s', y='Conc', c='YEAR',  alpha=0.5)
plt.colorbar(plt2, ax=axs[1, 0], orientation ='horizontal')

# Plotting in log-log space
axs[1, 1].scatter(x=np.log10(CQ_Site['Q_m3s']), y=np.log10(CQ_Site['Conc']), c=CQ_Site['YEAR'],alpha=0.5)

axs[1, 1].set_title('Neg. Power law b metric')
axs[0, 1].set_ylabel('log C')
axs[1, 1].set_ylabel('log C')
axs[1, 1].set_xlabel('log Q')

plt.tight_layout()

How do the watersheds differ? What are the signs of their β slopes?

Let's go ahead and derive β for all sites. 

In [ ]:
def fit_PowerLaw(group):
    X = np.log(group['Q_m3s']).values.reshape(-1, 1)
    y = np.log(group['Conc']).values
    model = LinearRegression()
    model.fit(X, y)
    group['b'] = model.coef_[0]
    return group

# Apply the log-transformed linear regression function to each group. Add as a columns.
CQ_attrib = CQ_attrib.groupby('SITE').apply(fit_PowerLaw, include_groups=False).reset_index(level=0)

In [ ]:
# Aggregate the attributes so each site only has 1 record.
CQ_attrib_mean = CQ_attrib.groupby('SITE').agg({'AREA_SQKM':'first', 
                                                'HUC02':'first', 
                                                'Q_m3s':'mean',
                                                'Conc': 'mean', 
                                                 'b': 'first',
                                                 'UrbLU_pct':'mean',
                                                 'NatLU_pct':'mean',
                                                 'AgLU_pct':'mean',
                                                 'FERT_kgkm2':'mean',
                                                 'MANU_kgkm2':'mean',
                                                 'BFI_AVE':'first',
                                                 'SLOPE_pct':'first',
                                                 'AI':'first',
                                                 'medianGWD_m':'first',
                                                 'TD_pct':'first',
                                                 'WWTPd_100km2':'first'}).reset_index()

### Spatial pattern CQ relationships

In [ ]:
# Start by looking at the spatial distribution of the 'b' metric
b_shp = wtshd_f.merge(CQ_attrib_mean, on='SITE')

plt.figure(figsize=(20,20))
base = US_boundary.plot(color='white', edgecolor='black')
b_shp.plot(ax=base,
           column='b', 
           cmap='RdYlBu',
           legend =  True,
           vmin=-1,
           vmax=1,
           edgecolor='none',
           legend_kwds={"label": "Power law 'b' metric", 
                        "orientation": "horizontal"})

base.set_xlim(-3*10**6, 3*10**6)
base.set_ylim(0, 3.5*10**6)

We can see that **enrichment** slope (positive b) is more common in the midwest. Outside of this region we see a lot more **dilution export**.

Doing another round of 'b metric' vs. watershed properties could help us understand the drivers of these export patterns. 

In [ ]:
# Which attributes would you like to plot?
CQ_attrib.columns

In [ ]:
# Let's take a look at how this looks with real data
plt.figure(figsize=(4, 4))

# Toggle the attributes
attribute = 'UrbLU_pct'

CQ_attrib_mean = CQ_attrib_mean.sort_values(by=[attribute])
plt.scatter(data=CQ_attrib_mean, x='Conc', y='b',c = attribute, alpha=0.5)
plt.colorbar(orientation ='horizontal')
plt.ylim(-10,10)
plt.ylabel('b')
plt.xlabel('Attribute')

Looking at β and different attributes, what can we say?
- Larger watershed...
- Natural land use...
- Agricultural land use...
- The β in agricultural land use plot and tile drainage plot look different? Similar?

#### Dimension Reduction to Understand Watershed Similarities
Understanding export behaviors with 12 different drivers and their interactions over 50 years can be challenging. To simplify this complexity, we can use dimension reduction techniques, specifically unsupervised machine learning.

We will apply k-means clustering to group events with similar export patterns.

The first step is to normalize the data, which involves selecting an appropriate method. Recall that many features have high extremes, which can pose issues for the k-means distance measure. To address this, we will set an upper limit on the values and then linearly transform them to fall between 0 and 1 for simplicity.

In [ ]:
# Function to normalize the data. First we will cap some of the data, then we will use linear scaling to put all the data between [0,1]
def linearNormalization(data, feature, min_val=None, max_val=None):
    if min_val is not None:
        data[feature] = np.maximum(data[feature], min_val)
    if max_val is not None:
        data[feature] = np.minimum(data[feature], max_val)
        
    min_val = data[feature].min()
    max_val = data[feature].max()
    data[feature] = (data[feature] - min_val) / (max_val - min_val)

In [ ]:
# Isolate the attributes we want to use in our clustering algorithm 
CQ_attrib['logAREA_SQKM'] = np.log10(CQ_attrib['AREA_SQKM'])
attributes =  ['Conc', 'logAREA_SQKM', 'UrbLU_pct', 'NatLU_pct', 'AgLU_pct', 'TD_pct', 'FERT_kgkm2',
               'MANU_kgkm2', 'BFI_AVE', 'SLOPE_pct', 'AI', 'medianGWD_m', 'WWTPd_100km2', 'b']
kMeans_data = CQ_attrib[attributes].copy()

# Setting upper limits to some of the data so we can apply a linear normalization without havies issues with large outliers.
max_values = {
    'Conc': 10,
    'FERT_kgkm2': 5000,
    'MANU_kgkm2': 5000,
    'SLOPE_pct':100, 
    'AI':2, 
    'medianGWD_m':30, 
    'WWTPd_100km2':1,
    'b':2}
min_values = {'b':-2}

# Normalizing the data
for feature in attributes:
    minVal = min_values.get(feature)
    maxVal = max_values.get(feature)
    linearNormalization(kMeans_data, feature, minVal, maxVal)

In [ ]:
# Checking the data to make sure it looks ok!
sns.pairplot(kMeans_data, corner=True)
sns.set(font_scale=5)

In [ ]:
# Reset defaults
plt.rcdefaults()

# We need to tune the k hyperparameter to find the optimal number of clusters. 
sse = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(kMeans_data)
    sse.append(kmeans.inertia_)

# Plotting the hyperparameter tuning error results
plt.figure(figsize=(4, 4))
plt.plot(range(1, 11, 1), sse, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Sum of squared distances (Inertia)')
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
# Choosing 'k' is a subjective choice. But 8 clusters looks like a good start. 
optimal_k = 8
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
kmeans.fit(kMeans_data)

# Add cluster labels to the original dataset
CQ_attrib['Cluster'] = kmeans.labels_
def cust_mode(series):
    return series.mode().iloc[0] if not series.mode().empty else np.nan

# Let's groupby the method based on site. We will take the MODE of the cluster for each site because it's a categorical data. 
CQ_attrib_modeClust = CQ_attrib.groupby('SITE').agg({'logAREA_SQKM':'first', 
                                            'HUC02':'first', 
                                            'Q_m3s':'mean',
                                            'Conc': 'mean', 
                                            'b': 'first',
                                            'UrbLU_pct':'mean',
                                            'NatLU_pct':'mean',
                                            'AgLU_pct':'mean',
                                            'FERT_kgkm2':'mean',
                                            'MANU_kgkm2':'mean',
                                            'BFI_AVE':'first',
                                            'SLOPE_pct':'first',
                                            'AI':'first',
                                            'medianGWD_m':'first',
                                            'TD_pct':'first',
                                            'WWTPd_100km2':'first',
                                            'Cluster':cust_mode}
                                            ).reset_index()

CQ_attrib_modeClust['Cluster'] = CQ_attrib_modeClust['Cluster'].astype('category')
CQ_attrib_modeClust.head()

#### Using the clusters to generalize the different behaviours

In [ ]:
# We are going to plot the bar plots of each clusters for each variable. 
CQ_attrib_long = pd.melt(CQ_attrib_modeClust, id_vars='Cluster', 
                         value_vars=['Conc', 'logAREA_SQKM', 'b', 'UrbLU_pct', 
                                     'NatLU_pct', 'AgLU_pct', 'FERT_kgkm2', 'MANU_kgkm2', 
                                     'BFI_AVE', 'SLOPE_pct', 'AI', 'medianGWD_m', 
                                     'TD_pct', 'WWTPd_100km2'],
                         var_name='Attribute', value_name='Value')

attributes = ['logAREA_SQKM', 'Conc', 'UrbLU_pct', 
              'NatLU_pct', 'AgLU_pct', 'TD_pct', 'FERT_kgkm2', 'MANU_kgkm2', 
              'BFI_AVE', 'SLOPE_pct', 'AI', 'medianGWD_m', 'WWTPd_100km2','b']

# Number of rows and columns for subplots
n_rows = 4
n_cols = 7

# Manually setting color palette to match the map's cmap. 
palette = [
    (0.27, 0.004, 0.33),
    (0.27, 0.20, 0.49),
    (0.21, 0.36, 0.55),
    (0.15, 0.50, 0.56),
    (0.12, 0.63, 0.53),
    (0.29, 0.76, 0.43),
    (0.63, 0.85, 0.22),
    (0.99, 0.91, 0.15)
]

# Using GridSpec to organize the figure to have the boxplots at the top and map at the bottom. 
fig = plt.figure(figsize=(18, 12))
gs = gridspec.GridSpec(n_rows, n_cols, figure=fig, height_ratios=[1,1, 1, 1])

# Plot each attribute in a separate subplot
for i, attribute in enumerate(attributes):
    row = i // n_cols
    col = i % n_cols
    ax = fig.add_subplot(gs[row, col])
    sns.boxplot(x='Cluster', 
                y='Value', 
                data=CQ_attrib_long[CQ_attrib_long['Attribute'] == attribute], 
                ax=ax, 
                showfliers=False, 
                palette=palette,
                hue='Cluster',
                dodge=False,
               legend=False)
    ax.set_title(attribute)
    ax.set_xlabel('Cluster')
    
plt.tight_layout()

# Merge datasets for spatial plot
b_shp = wtshd_f.merge(CQ_attrib_modeClust, on='SITE')

# Create the subplot for the map, occupying the entire last row
ax_map = fig.add_subplot(gs[2:, :])
base = US_boundary.plot(ax=ax_map, color='white', edgecolor='black')
b_shp.plot(ax=base,
           column='Cluster', 
           cmap='viridis',
           legend=True)
ax_map.set_xlim(-3*10**6, 3*10**6)
ax_map.set_ylim(0, 3.5*10**6)
ax_map.set_title('Spatial Distribution of the clusters')

plt.show()

## What do we see in our cluster analysis?
We are starting to see some patterns emerge once we are able to apply dimension reduction. 

**Clusters 0**: Agricultural watersheds that are heavily tile-drained. These watersheds have high concentrations and highest fertilizer and manure inputs. These watersheds have a lot of nitrogen and thus have a lot extra to export. 

**Cluster 1**: Arid watersheds with deep groundwater and much of the watershed is natural land use. These watersheds have low concentration and high base flow index. 

**Cluster 2**: Highly urban watersheds but with low wasterwater treatment plants density. This explains why we might not see the dilution pattern expected with wastewater treatment plants. 

**Cluster 3**: Agricultural watersheds that are not heavily tile-drained. These watersheds have a more constant export, in comparison to Cluster 0.

**Cluster 4**: Agricultural watersheds that are moderately tile-drained. Spatially they surround Cluster 0 watersheds, so they are similar but are less influenced by the effects of tile drainage on export. 

**Cluster 5**: Urban watersheds with more higher wastewater treatment density. Median b metric is negative, which means these watersheds tend to have more of a dilution patterns. These watersheds tend to be smaller, which is perhaps why we can see the effects of urban nitrogen sources on the concentration.

**Cluster 6**: Similar to cluster 1 but in temperate climates (low aridity index). 

**Cluster 7**: Arid agricultural watersheds. These watersheds have similar properties to Cluster 0 and Cluster 3, but because they are arid the groundwater residence time is much longer, thus we are seeing very different signals compared to more temperate agricultural watersheds. 

### Final Thoughts on Spatial Patterns
Using a series of approaches, we were able to abstract and find some patterns in our dataset. The b metric helped us a _little_, but there isn't much variation across the clusters. That's okay! Explorers don't always strike gold right away! 

What are the next steps, you might ask? If this were my project, I would return to the feature engineering step and see what other type of features I can generate to explore the effects of watershed properties on C-Q relationships. Additionally, after seeing some interesting patterns from our analysis, I might try to find other watershed attributes to dig deeper into some of the threads I've created. 

### Temporal pattern CQ relationships
We will not be exploring CQ relationships over time. But for those who are interested in digging further or who are experienced coders, I encourage you to explore the temporal patterns of CQ relationships. Here are some prompt to help get your started:
- How has 'β' metric change over time? Split the data by decade and track the changes in the watershed.
- Do we see the influence of changing land use or inputs (Fertilizer or manure) on this relationship? 
- Do we see annual variability in the 'β' metric in temporate or arid catchments? Do we see more annual variability in hydrologically connected landscapes? Does the slow movement of arid system act as a modulator of changing relationships?
- How does CQ relationship change seasonally?